In [ ]:
from os.path import abspath

from pyspark.sql import SparkSession
from  pyspark.sql.catalog import Catalog
from pyspark.sql.functions import *
from pyspark.sql.types import *

In [ ]:
#CRIANDO SESSÃO

warehousePath = abspath('spark_database')
sparkv2path = '/home/doug/ProjetosEstudo/LearningSparkV2/databricks-datasets/learning-spark-v2'
sparkdefguidepath = '/home/doug/ProjetosEstudo/Spark-The-Definitive-Guide'
master= "local[*]"
worker="spark://DOUGPC.:7077"

spark = SparkSession.\
        builder.\
        appName('SparkSQL').\
        master(master).\
        config("spark.sql.warehouse.dir", warehousePath).\
        config("spark.sql.catalogImplementation", "hive").\
        config("spark.sql.legacy.createHiveTableByDefault", "false").\
        enableHiveSupport().\
        getOrCreate()
        
print(warehousePath)

In [ ]:
#RODANDO UMA QUERY EM SQL COM PYTHON
df = spark.sql("""select 1+1""")
df.show(1)
parquetPath = (f'{sparkdefguidepath}/data/flight-data/parquet/2010-summary.parquet')

print(sparkdefguidepath)


In [ ]:
schemaFlight = StructType([
                            StructField('DEST_COUNTRY_NAME', StringType(), nullable= False),
                            StructField('ORIGIN_COUNTRY_NAME', StringType(), nullable= False),
                            StructField('count', IntegerType(), nullable= False)])

flightframe = spark.read.csv(f'{sparkdefguidepath}/data/flight-data/csv/2010-summary.csv', schema= schemaFlight, header=True, sep=',').\
                        withColumnRenamed('DEST_COUNTRY_NAME', 'destine').\
                        withColumnRenamed('ORIGIN_COUNTRY_NAME', 'origin').\
                        withColumnRenamed('count', 'flyqtd')

#img01 erro dataframe  com sql                      
flightframe = spark.sql("""
                        SELECT 
                            origin,
                            flyqtd
                        FROM flightframe
                        where flyqtd > 100""")

flightframe.show()

In [ ]:
#img02 criando uma view temporária e executando uma consulta sql                        
flightframe.createOrReplaceTempView('flightview')

viewFrame = spark.sql("""
                select origin, flyqtd
                from flightview
                where flyqtd * 2 > flyqtd""")

viewFrame.show(3)

#img02.1 group by com pyspark 
viewFrame.groupBy('origin').\
            sum('flyqtd').\
            show(3)


            
#manipulando um dataframe com SQL e com a API PySpark

doubleWhrere = spark.sql("""
                        select 
                            ORIGIN,
                            FLYQTD
                        FROM FLIGHTVIEW
                        WHERE FLYQTD > 100""").\
                    where("origin like 'I%'").\
                    where("flyqtd > 200")

doubleWhrere.show(3)
            

In [ ]:

windowFrame = spark.sql("""
                        select 
                            origin,
                            flyqtd,
                            row_number() over (order by origin) as rownum
                        from flightview
                        where flyqtd > 100""")
windowFrame.show(3)



partitionFrame = spark.sql("""
                            SELECT
                                ORIGIN,
                                FLYQTD,
                                SUM(FLYQTD) OVER (PARTITION BY ORIGIN
                                                    ORDER BY FLYQTD
                                                    ROWS UNBOUNDED PRECEDING) as SumPerOrigin
                            FROM FLIGHTVIEW""")
partitionFrame.show(3)

In [ ]:
delayFlightSchema = StructType([
                                StructField('date', StringType(), nullable= False),
                                StructField('delay', IntegerType(), nullable= False),
                                StructField('distance', IntegerType(), nullable= False),
                                StructField('origin', StringType(), nullable= True),
                                StructField('destination', StringType(), nullable= False)])


flightDelayPath = spark.read.csv(f'{sparkv2path}/flights/departuredelays.csv', header= True, schema= delayFlightSchema)

flightDelayPath.createOrReplaceTempView('tempflight')

flightViewNoReplace = spark.read.csv(f'{sparkv2path}/flights/departuredelays.csv', header= True, schema= delayFlightSchema)

# flightViewNoReplace.createTempView('noreplaceflight')

castDate = spark.sql("""
                    SELECT
                        DATE,
                        SUBSTRING(DATE, 0, 2) AS SUB,
                        SUBSTRING(DATE, 3, 2) AS SUB2,
                        DELAY,
                        ORIGIN,
                        DESTINATION
                    FROM TEMPFLIGHT
                    ORDER BY DELAY DESC""")

concatview = castDate.createOrReplaceTempView('castDate')

# concatws = spark.sql("""
#                      select
#                         sub,
#                         sub2,
#                         concat_ws('-', sub, sub2) as date
#                     from castDate""")
# concatws.show(2)

tempFrame = castDate.select(concat_ws('-', castDate.SUB, castDate.SUB2).alias('DATE'),
                castDate.DELAY,
                castDate.ORIGIN,
                castDate.DESTINATION)

tempFrame.createOrReplaceTempView('usflight')
tempFrame.show()

In [ ]:
# criando uma classificação com SQL CASE

classfication = spark.sql("""
                            SELECT
                                DATE, ORIGIN, DESTINATION, DELAY, 
                                CASE
                                    WHEN DELAY > 360 THEN 'VERY LONG'
                                    WHEN DELAY >= 120 AND DELAY < 360 THEN 'LONG DELAY'
                                    WHEN DELAY >= 60 AND DELAY < 120 THEN 'SHORT DELAY'
                                    WHEN DELAY > 0 AND DELAY < 60 THEN 'TOLERABLE'
                                    WHEN DELAY = 0 THEN 'NO DELAY'
                                ELSE 'EARLY'
                            END AS US_DELAY
                            FROM usflight""")

classfication.show()

In [ ]:
#criando cte com case
classfication = spark.sql("""
                            with class_flight as (
                                select
                                    DATE, ORIGIN, DESTINATION, DELAY, 
                                    CASE
                                        WHEN DELAY > 360 THEN 'VERY LONG'
                                        WHEN DELAY >= 120 AND DELAY < 360 THEN 'LONG DELAY'
                                        WHEN DELAY >= 60 AND DELAY < 120 THEN 'SHORT DELAY'
                                        WHEN DELAY > 0 AND DELAY < 60 THEN 'TOLERABLE'
                                        WHEN DELAY = 0 THEN 'NO DELAY'
                                    ELSE 'EARLY'
                                END AS US_DELAY
                                FROM usflight)
                                
                            select * from class_flight
                            where US_DELAY LIKE "TOLE%"
                            """)
classfication.show()


In [ ]:
tempFrame.select('DELAY', 'ORIGIN', 'DESTINATION').\
            where('DELAY > 1500').\
            orderBy('DELAY', ascending= False).show()
            
tempFrame.select(concat('DELAY', 'DESTINATION').\
                    alias('DELAY_DEST'), 'ORIGIN').\
                show(5)

In [ ]:
#IMG06

tempFrame.select('DELAY', 'ORIGIN').\
            orderBy('ORIGIN', ascending= False).\
            groupBy('ORIGIN').\
            sum('DELAY').\
        show(5)

In [ ]:
spark.sql( """ drop database if exists testedb""")

spark.sql (""" CREATE DATABASE testeDB """)

In [ ]:
#CRIANDO TABELAS GERENCIADAS

#img07

spark.sql("""DROP DATABASE IF EXISTS flightdb CASCADE""")

spark.sql("""CREATE DATABASE flightdb""")

spark.sql(""" USE flightdb """)

spark.sql(""" DROP TABLE IF EXISTS FLIGHTTABLE""")

pathSource = f'{sparkv2path}/flights/departuredelays.csv'

flightTableSchema = StructType([
                                StructField("date", StringType(), False),
                                StructField("delay", IntegerType(), False),
                                StructField("distance", IntegerType(), False),
                                StructField("origin", StringType(), False),
                                StructField("destination", StringType(), False)])

flightTable = spark.read.csv(path=pathSource,
                             header= True,
                             schema= flightTableSchema)

#criando uma tabela gerenciada.
#utiliza o método write antes do saveAsTable
flightTable.write.saveAsTable(name="FlightTable", mode="overwrite")

selectTable = spark.sql("""
                        SELECT * FROM FLIGHTTABLE """)

selectTable.show(1)

In [ ]:
#IMG 7.1 TABELA GERENCIADA COM SPARK.SQL
#IMG 7.2 PRINT DOS ARQUIVOS NO DIRETÓRIO.

spark.sql("""USE flightdb """)

#para criar tabelas gerenciadas, armazenando tanto os dados quanto os metadados

spark.sql(""" DROP TABLE IF EXISTS USFLIGHT """)

spark.sql ("""
            
            CREATE TABLE USFLIGHT(
                
                DATE STRING,
                DELAY INT,
                DISTANCE INT,
                ORIGIN STRING,
                DESTINY STRING
                
            )""")#.explain(mode= 'formatted')
#inserindo
spark.sql(""" 
          INSERT INTO USFLIGHT (DATE, DELAY, DISTANCE, ORIGIN, DESTINY)
          VALUES (1111111, 111111, 111111, 'AAAAA', 'BBBBB')""")

#consultando
spark.sql("""
          SELECT *
          FROM USFLIGHT 
          WHERE ORIGIN LIKE 'AAA%'""").show()


In [ ]:
#CRIANDO UMA TABELA NÃO GERENCIADA
#VEJA QUE UTILIZO O USING COM O FORMATO DO ARQUIVO E AS OPÇÕES.
#COMO SE ESTIVESSE UTILIZANDO A PYSPARK API.

#IMG08
#IMG08.1 PRINT COM O PATH DO SPARK_DATABASE SEM A TABELA CRIADA.
spark.sql(""" USE flightdb """)

spark.sql(""" drop table if exists sales_table""")

spark.sql(""" 
        
        CREATE TABLE SALES_TABLE (
            INVOICE_NO STRING,
            STOCKCODE STRING,
            DESCRIPTION STRING,
            QUANTITY INT,
            INVOICEDATE STRING,
            UNITPRICE DECIMAL,
            CUSTOMERID INT,
            COUNTRY STRING)
           
        USING csv OPTIONS (header true, 
        PATH '/home/doug/ProjetosEstudo/Spark-The-Definitive-Guide/data/retail-data/all/online-retail-dataset.csv') """)

spark.sql(""" SELECT * FROM SALES_TABLE""").show(2)

In [ ]:
spark.sql(""" drop table if exists teste_table""")

spark.sql("""
          CREATE TABLE TESTE_TABLE 
          USING parquet OPTIONS
          (PATH "/home/doug/ProjetosEstudo/Spark-The-Definitive-Guide/data/flight-data/parquet/2010-summary.parquet")""")

spark.sql ("select * from teste_table""").\
        explain(mode= "formatted") 

In [ ]:

parquetFrame = spark.read.parquet(f"{sparkdefguidepath}/data/flight-data/parquet/2010-summary.parquet").\
                        withColumnRenamed("DEST_COUNTRY_NAME", "destine").\
                        withColumnRenamed("ORIGIN_COUNTRY_NAME", "origin")
                        
parquetFrame.createOrReplaceTempView("prqFlight")

# spark.sql(""" select * from prqflight""").show(2)
spark.sql(""" USE flightdb """)

spark.sql(""" drop table if exists flight""")

spark.sql("""
            CREATE TABLE FLIGHT               
            USING parquet 
            PARTITIONED BY (destine) AS 
            SELECT * 
            FROM prqFlight
            ORDER BY ORIGIN
            
            """)

spark.sql(""" select * from flight""").show()

In [ ]:
#CRIANDO TABELA GERENCIADA COM CREATE EXTERNAL TABLE

spark.sql( """ drop table if exists external_table """)

spark.sql( """ 
            create external table external_table
            row format delimited fields terminated by ','
            location '/home/doug/ProjetosEstudo/PySpark/SparkSQL/spark_database/external/'
            as select * from flight """).\
        explain(mode= 'formatted')

spark.sql (""" select * from external_table """).\
        explain(mode= 'formatted')
        
# spark.sql(""" select * from external_table """).show(10)

In [ ]:
#CRIANDO TABELAS COM "SELECT INTO"
#DEVE SER CRIADA A PARTIR DE UMA VIEW OU TABLE EXISTENTE
#LEMBRANDO QUE É UMA TABELA GERENCIADA

#tempflight é uma view criada no início deste notebook

spark.sql(""" drop table if exists dtwtable""")

spark.sql(""" CREATE TABLE DTWTABLE 
                SELECT * FROM tempflight 
                WHERE destination = 'DTW' """)



In [ ]:
salesTable = spark.sql( """ select * from sales_table """)

spark.sql(""" DROP TABLE IF EXISTS SALESGREATER10""")

#IMG08.2.1 CRIANDO UMA TABELA GERENCIADA UTILIZANDO PYSPARK + DATAFRAME
salesTable.select('INVOICE_NO','QUANTITY', 'UNITPRICE', 'CUSTOMERID').\
            where("COUNTRY like 'United%'").\
            where("QUANTITY >= 10").\
        write.\
            saveAsTable("salesGreater10")
            
#consultando a tabela            
spark.sql(""" SELECT * FROM SALESGREATER10""").show(5)


In [ ]:
# flightframe.show(1)

spark.sql(""" drop table if exists explain_table""")

flightframe.write.saveAsTable('explain_table')

spark.sql("""  select * from explain_table """).\
        explain(mode= "formatted")
        
print(warehousePath)

In [ ]:
#TABELA PARTICIONADA

#CRIANDO UMA TABELA PARTICIONADA A PARTIR DE UMA TABELA N-GERENCIADA
#IMG8.3 TABELA PARTICIONADA A PARTIR DE UMA TABELA N-GERENCIADA.

# spark.sql(""" DROP TABLE IF EXISTS partSALESTABLE """)

# spark.sql(""" CREATE TABLE partSALESTABLE
#           PARTITIONED BY (CUSTOMERID)
#           AS 
#           SELECT *
#           FROM SALES_TABLE """)

# spark.sql(""" 
#             SELECT * 
#             FROM partSALESTABLE
#             WHERE CUSTOMERID IS NOT NULL""").\
#       show(1)
      
#PRINT 8.4 CRIANDO VIEW PARTICIONADA VIA WINDOW FUNCTION
spark.sql( """  
           SELECT 
            INVOICE_NO,
            STOCKCODE,
            QUANTITY,
            CUSTOMERID,
           row_number() OVER(PARTITION BY CUSTOMERID
                              ORDER BY CUSTOMERID
                              ROWS UNBOUNDED PRECEDING) AS RN
           FROM SALES_TABLE """).\
      createOrReplaceTempView('partview')

spark.sql(""" select * from partview """).\
      show(2)

In [ ]:
#print 8.5 utilizando describe 

#print 8.5.1 criando a tabela e consultando
spark.sql(""" describe table flight """)

spark.sql(""" drop table if exists metadata_salesgreater """)

spark.sql( """ describe table salesgreater10 """).\
        write.\
        saveAsTable('metadada_salesgreater', mode= 'append')
        
spark.sql(""" select * from metadada_salesgreater """ ).show()

In [ ]:
#print 8.5.2 partição, atualização dos metadados e reparo das tabelas

spark.sql(""" show partitions partsalestable """).show(2)

spark.sql(""" refresh table usflight """).show(5)

spark.sql(""" msck repair table partsalestable """).show(5) 

In [ ]:
#cache table
#print 8.6 cache table
spark.sql(""" cache table partsalestable """)

spark.sql(""" uncache table partsalestable """)

#print 8.6.1 cache table na criação
spark.sql(""" drop table if exists cache_table """)

spark.sql(""" create table cache_table
                           SELECT 
            INVOICE_NO,
            STOCKCODE,
            QUANTITY,
            CUSTOMERID,
           row_number() OVER(PARTITION BY CUSTOMERID
                              ORDER BY CUSTOMERID
                              ROWS UNBOUNDED PRECEDING) AS RN
           FROM SALES_TABLE """
                                ).cache()


In [ ]:
spark.sql(""" show tables in  flightdb """).show()

#execute os comandos abaixo para excluir os databases, tabelas e views.
# spark.sql(""" drop database if exists flight_db cascade """)

# spark.sql(""" drop database if exists teste_db cascade """)

# spark.sql(""" drop table if exists external """)



In [ ]:
# spark.sql(""" drop table cache_table """)
# spark.sql(""" drop table dtwtable """)
# spark.sql(""" drop table explain_table """)
# spark.sql(""" drop table flight """)
# spark.sql(""" drop table flighttable """)
# spark.sql(""" drop table metadata_salesgreater """)
# spark.sql(""" drop table partsalestable """)
# spark.sql(""" drop table salesgreater10 """)
# spark.sql(""" drop table usflight """)